In [43]:
import numpy as np
import os
import scipy.sparse as sp

from spcqe.functions import make_basis_matrix, make_regularization_matrix
from spcqe.test_functions import basis_3, basis_2

In [31]:
directory = 'fixtures'

## Basis matrix

3 periods and a trend term

In [32]:
basis_a1 = make_basis_matrix(num_harmonics=10, length=500, periods=[11, 17, 23], trend=True)
np.save(os.path.join(directory, 'basis_a1.npy'), basis_a1)

2 periods and no trend term

In [33]:
basis_a2 = make_basis_matrix(num_harmonics=10, length=400, periods=[11, 17], trend=False)
np.save(os.path.join(directory, 'basis_a2.npy'), basis_a2)

multiple harmonics and standing wave

In [34]:
basis_a3 = make_basis_matrix(num_harmonics=[3, 7], length=370, periods=[11, 3], standing_wave=[True, False], trend=False)
np.save(os.path.join(directory, 'basis_a3.npy'), basis_a3)

multiple harmonics and standing wave and max cross terms

In [35]:
basis_a4 = make_basis_matrix(num_harmonics=[4, 3], length=420, periods=[6, 8], standing_wave=True, trend=True, max_cross_k=4)
np.save(os.path.join(directory, 'basis_a4.npy'), basis_a4)

## Regularization matrix

3 periods and a trend term

In [45]:
reg_a1 = make_regularization_matrix(num_harmonics=10, weight=10, periods=[11, 17, 23], trend=True)
sp.save_npz(os.path.join(directory, 'reg_a1.npz'), reg_a1)

2 periods and no trend term

In [47]:
reg_a2 = make_regularization_matrix(num_harmonics=10, weight=1, periods=[11, 17], trend=False)
sp.save_npz(os.path.join(directory, 'reg_a2.npz'), reg_a2)

multiple harmonics and standing wave

In [48]:
reg_a3 = make_regularization_matrix(num_harmonics=[3, 7], weight=0, periods=[11, 3], standing_wave=[True, False], trend=False)
sp.save_npz(os.path.join(directory, 'reg_a3.npz'), reg_a3)

multiple harmonics and standing wave and max cross terms

In [49]:
reg_a4 = make_regularization_matrix(num_harmonics=[4, 3], weight=3, periods=[6, 8], standing_wave=True, trend=True, max_cross_k=4)
sp.save_npz(os.path.join(directory, 'reg_a4.npz'), reg_a4)